# Step 2-1. 벡터 유사도 검색
로컬 임베딩 모델을 사용하여 k-NN 유사도 검색을 수행합니다.

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0 sentence-transformers==4.1.0

## 1. 설정 (Configuration)

In [ ]:
import os, json

# Step 0에서 저장한 설정 불러오기
try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    raise FileNotFoundError("❌ config.json을 찾을 수 없습니다. Step 0 노트북을 먼저 실행해주세요.")

HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

## 2. OpenSearch 클라이언트 생성

In [2]:
import boto3
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

service = 'aoss'
credentials = boto3.Session(profile_name=PROFILE).get_credentials()
auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, service)

client = OpenSearch(
    hosts=[{'host': HOST, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

print("OpenSearch 클라이언트 생성 완료")

OpenSearch 클라이언트 생성 완료


## 3. 임베딩 모델 로드 및 검색어 벡터 변환

In [ ]:
from sentence_transformers import SentenceTransformer

INDEX_NAME = 'vector-test'
EMBEDDING_MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

print(f"Loading embedding model: '{EMBEDDING_MODEL_NAME}'...")
model = SentenceTransformer(EMBEDDING_MODEL_NAME)
print("Model loaded successfully.")

In [ ]:
# 인덱스에 데이터가 업로드되었는지 확인
if not client.indices.exists(index=INDEX_NAME):
    print(f"❌ '{INDEX_NAME}' 인덱스가 존재하지 않습니다.")
    print("   → Step 2-0 (벡터 데이터 업로드) 노트북을 먼저 실행해주세요.")
else:
    _doc_count = client.count(index=INDEX_NAME)['count']
    if _doc_count == 0:
        print(f"⏳ '{INDEX_NAME}' 인덱스는 있지만 검색 가능한 문서가 0개입니다.")
        print("   → 데이터 업로드 직후라면 인덱싱 중일 수 있습니다. 잠시 후 다시 실행해주세요.")
    else:
        print(f"✅ '{INDEX_NAME}' 인덱스에 {_doc_count}개의 문서가 준비되어 있습니다.")

## 4. k-NN 검색 실행 및 결과 출력

In [ ]:
# TODO: 검색할 문장을 바꿔보세요!
QUERY_TEXT = "s3랑 cloudfront를 활용해서 정적 웹사이트를 배포하는 방법 알려줘"

print(f"Creating a vector for the query: '{QUERY_TEXT}'...")
query_vector = model.encode(QUERY_TEXT).tolist()

K_NEIGHBORS = 5
RESULT_SIZE = 5

search_query = {
    "size": RESULT_SIZE,
    "query": {
        "knn": {
            "content_vector": {
                "vector": query_vector,
                "k": K_NEIGHBORS
            }
        }
    }
}

print(f"Searching for top {RESULT_SIZE} similar documents...")
response = client.search(index=INDEX_NAME, body=search_query)

hits = response['hits']['hits']
print(f"\n--- '{QUERY_TEXT}'와(과) 유사한 문서 검색 결과({len(hits)}) ---")

if not hits:
    print("검색된 문서가 없습니다.")
else:
    for i, hit in enumerate(hits):
        score = hit['_score']
        title = hit['_source'].get('title', 'N/A')
        content = hit['_source'].get('content', 'N/A')
        print(f"\n[{i+1}] 유사도: {score:.4f}")
        print(f"    제목: {title}")
        print(f"    내용: {content[:100]}...")

## 실습 과제

### 과제 1: 다운로드된 임베딩 모델 찾아보기
모델은 `~/.cache/huggingface/hub/` 경로에 다운로드됩니다. 아래 셀을 실행하여 확인해보세요.

### 과제 2: 다른 한국어 모델로 검색 결과 비교하기
한국어를 지원하는 다른 Sentence Transformer 모델을 찾아서 적용해보세요.
- 인덱스 이름은 `vector-test2`로 설정
- step2/0 노트북에서 모델과 인덱스 이름을 변경하여 데이터를 업로드한 후, 이 노트북에서 검색 결과를 비교
- 참고: [HuggingFace Sentence Transformers](https://huggingface.co/models?library=sentence-transformers&language=ko)

In [5]:
# 과제 1: 다운로드된 임베딩 모델 확인
import os
cache_path = os.path.expanduser("~/.cache/huggingface/hub/")
if os.path.exists(cache_path):
    models = [d for d in os.listdir(cache_path) if d.startswith("models--")]
    print(f"다운로드된 모델 목록 ({len(models)}개):")
    for m in models:
        print(f"  - {m.replace('models--', '').replace('--', '/')}")
else:
    print(f"캐시 경로가 존재하지 않습니다: {cache_path}")

다운로드된 모델 목록 (1개):
  - sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
